In [1]:
!pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
from datetime import datetime

def amazon_a_camel_url(amazon_url: str) -> str:
    """Convierte una URL de Amazon a una URL de CamelCamelCamel usando el ASIN."""
    asin_match = re.search(r'/dp/([A-Z0-9]{10})', amazon_url)
    if asin_match:
        asin = asin_match.group(1)
        camel_url = f"https://camelcamelcamel.com/product/{asin}"
        return camel_url
    else:
        raise ValueError("No se pudo extraer el ASIN de la URL de Amazon")

def extraer_precio_y_fecha(texto: str):
    """Extrae el precio y la fecha del texto proporcionado."""
    precio_match = re.search(r'\$(\d+\.\d{2})', texto)
    fecha_match = re.search(r'\((\d{2} de \w+ de \d{4})\)', texto)
    
    precio = float(precio_match.group(1)) if precio_match else None
    fecha_str = fecha_match.group(1) if fecha_match else None
    
    # Mapear los meses abreviados en español
    meses = {
        'ene': '01',
        'feb': '02',
        'mar': '03',
        'abr': '04',
        'may': '05',
        'jun': '06',
        'jul': '07',
        'ago': '08',
        'sep': '09',
        'oct': '10',
        'nov': '11',
        'dic': '12'
    }
    
    if fecha_str:
        for mes_abrev, mes_num in meses.items():
            fecha_str = fecha_str.replace(f" de {mes_abrev} de ", f"-{mes_num}-")
        fecha = datetime.strptime(fecha_str, '%d-%m-%Y')
    else:
        fecha = None
    
    return precio, fecha

# Ejemplo de URL de Amazon
url_amazon: str = "https://www.amazon.com/dp/B000VXO4L2?tag=camelweb-20&linkCode=ogi&th=1&psc=1&language=en_US"

# Convertir a URL de CamelCamelCamel
url: str = amazon_a_camel_url(url_amazon)

# Configurar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Iniciar el navegador Chrome
driver = webdriver.Chrome(service=service)

# Navegar a la página de CamelCamelCamel
driver.get(url)

# Buscar la tabla que contiene los precios
tabla_precios = driver.find_element(By.CLASS_NAME, 'camelegend')

# Extraer los datos de la tabla
datos: list[list] = []
filas = tabla_precios.find_elements(By.TAG_NAME, 'tr')
for fila in filas[1:]:  # Omitir la cabecera de la tabla
    celdas = fila.find_elements(By.TAG_NAME, 'td')
    tipo_precio = celdas[0].text.strip()
    
    mas_bajo_precio, mas_bajo_fecha = extraer_precio_y_fecha(celdas[1].text.strip())
    mas_alto_precio, mas_alto_fecha = extraer_precio_y_fecha(celdas[2].text.strip())
    actual_precio, actual_fecha = extraer_precio_y_fecha(celdas[3].text.strip())
    
    media_precio = float(re.search(r'\$(\d+\.\d{2})', celdas[4].text.strip()).group(1)) if celdas[4].text.strip() else None
    
    datos.append([tipo_precio, mas_bajo_precio, mas_bajo_fecha, mas_alto_precio, mas_alto_fecha,
                  actual_precio, actual_fecha, media_precio])

# Crear el DataFrame con las nuevas columnas
df = pd.DataFrame(datos, columns=['tipo_de_precio', 'mas_bajo', 'mas_bajo_fecha',
                                  'mas_alto', 'mas_alto_fecha', 'actual', 'actual_fecha', 'media'])

# Cerrar el navegador
driver.quit()

In [13]:
# Mostrar datos al usuario
print(df)

# Guardar los datos en un archivo CSV
df.to_csv('historial_precios.csv', index=False)

   tipo_de_precio  mas_bajo mas_bajo_fecha  mas_alto mas_alto_fecha  actual  \
0          Amazon      5.48     2020-02-11     21.49     2022-04-24    8.27   
1  3ª Parte Nuevo      0.99     2019-08-03     39.90     2022-04-21     NaN   
2  3ª Parte Usado      3.98     2016-03-09     53.32     2013-05-30     NaN   

  actual_fecha  media  
0   2024-08-25   9.93  
1          NaT  15.99  
2          NaT   9.47  


In [15]:
df

,tipo_de_precio,mas_bajo,mas_bajo_fecha,mas_alto,mas_alto_fecha,actual,actual_fecha,media
0,Amazon,5.48,2020-02-11,21.49,2022-04-24,8.27,2024-08-25,9.93
1,3ª Parte Nuevo,0.99,2019-08-03,39.90,2022-04-21,NaN,NaT,15.99
2,3ª Parte Usado,3.98,2016-03-09,53.32,2013-05-30,NaN,NaT,9.47


In [19]:
nuevo_df = df[df['tipo_de_precio'] == 'Amazon']

In [21]:
nuevo_df

,tipo_de_precio,mas_bajo,mas_bajo_fecha,mas_alto,mas_alto_fecha,actual,actual_fecha,media
0,Amazon,5.48,2020-02-11,21.49,2022-04-24,8.27,2024-08-25,9.93


In [23]:
nuevo_df.to_csv('historial_precios_solo_amazon.csv', index=False)